# Architecture B — multi-agent single-model

Notebook per eseguire il grafo multi-ruolo con lo stesso modello (`Qwen/Qwen2.5-Coder-7B-Instruct`) su Colab. Clona la repo, configura HF Inference, costruisce il grafo e salva i log in `log/`.

Linee guida:
- Fornisci `HF_TOKEN` quando richiesto.
- Esegui pochi task APPS per limitare costi.
- I log vengono scritti sia su stdout sia in file JSONL/LOG.

In [ ]:
import os
import sys
import subprocess
import pathlib

REPO_URL = "https://github.com/LLM4SE-group-15/ArchitecturesForCodeDevelopmentWithLLMs.git"
REPO_DIR = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

os.chdir(REPO_DIR)
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "pip"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"], check=True)

print(f"Using repo at {REPO_DIR.resolve()}")

In [ ]:
import os
import getpass

from huggingface_hub import login

# Always provide the token explicitly:
# - avoids Colab "Secrets" timeouts when running via VS Code extension
# - ensures gated models (if used) can be accessed

if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter HF_TOKEN (kept hidden): ")

login(token=os.environ["HF_TOKEN"], add_to_git_credential=False)

os.environ["ARCHITECTURE"] = "B"
print("ARCHITECTURE set to", os.environ["ARCHITECTURE"])

In [ ]:
import json
import time
import logging
import pathlib

DEFAULT_ROOT = pathlib.Path("/content/ArchitecturesForCodeDevelopmentWithLLMs")
ROOT = DEFAULT_ROOT if DEFAULT_ROOT.exists() else pathlib.Path.cwd()

sys.path.insert(0, str(ROOT))

LOG_DIR = ROOT / "log"
LOG_DIR.mkdir(exist_ok=True)

logger = logging.getLogger("architecture_B")
logger.setLevel(logging.INFO)
if logger.handlers:
    logger.handlers.clear()
formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
file_handler = logging.FileHandler(LOG_DIR / "architecture_B.log")
stream_handler = logging.StreamHandler()
for handler in (file_handler, stream_handler):
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Logger ready. Repo root: %s", ROOT)
logger.info("Log files: %s", LOG_DIR)
print("Logs ->", LOG_DIR)

In [ ]:
from src.data.task_loader import APPSTaskLoader
from src.graph.graph import run_graph
from src.agents.llm import Architecture

ARCH = Architecture.B


def run_sample_tasks(per_level: int = 1, split: str = "test"):
    loader = APPSTaskLoader(split=split)
    tasks = loader.load_by_difficulty("introductory", limit=per_level)
    results = []
    for task in tasks:
        logger.info("Running %s (%s)", task.task_id, task.difficulty)
        start = time.time()
        state = run_graph(
            task_id=task.task_id,
            task_description=task.question,
            test_inputs=task.inputs,
            test_outputs=task.outputs,
            architecture=ARCH,
        )
        elapsed = time.time() - start
        record = {
            "task_id": task.task_id,
            "difficulty": task.difficulty,
            "architecture": str(ARCH.value),
            "test_passed": state["test_passed"],
            "developer_tier": state.get("developer_tier"),
            "escalations": state["escalations"],
            "story_points_initial": state.get("story_points_initial"),
            "story_points_final": state.get("story_points_current"),
            "elapsed_seconds": elapsed,
        }
        results.append(record)
        logger.info(
            "Finished %s | pass=%s tier=%s escalations=%s elapsed=%.1fs",
            task.task_id,
            state["test_passed"],
            record["developer_tier"],
            record["escalations"],
            elapsed,
        )
        with open(LOG_DIR / "architecture_B.jsonl", "a", encoding="utf-8") as f:
            f.write(json.dumps(record) + "\n")
    return results

sample_results = run_sample_tasks(per_level=1)
sample_results